# Speech Understanding
## Assignment 2
#### Question 1.3
### Akansha Gautam (M23CSA506)

# Import libraries

In [35]:
import numpy as np
import pandas as pd
import random
import torch
import subprocess
from transformers import Wav2Vec2Processor, HubertForCTC
from sklearn.metrics import roc_curve
from transformers import AutoModel, AutoProcessor, AutoFeatureExtractor
from speechbrain.inference.separation import SepformerSeparation as separator
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.io import wavfile as wav
from mir_eval.separation import bss_eval_sources
from pesq import pesq
import torch.nn.functional as F
import IPython.display as ipd
import librosa.display
import torchaudio
import torchaudio.transforms as transforms
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from transformers import Wav2Vec2Model, Wav2Vec2FeatureExtractor
from transformers import Wav2Vec2Processor, WavLMModel
from peft import get_peft_model, LoraConfig, TaskType
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier
from sklearn.mixture import GaussianMixture
from sklearn.metrics import confusion_matrix
from transformers import UniSpeechSatModel, UniSpeechSatConfig, Wav2Vec2FeatureExtractor
from scipy.interpolate import interp1d
from scipy.optimize import brentq
from tqdm import tqdm
from sklearn.metrics.pairwise import cosine_similarity
from pytube import YouTube
import ffmpeg
import os
import loralib as lora
from loralib import Linear as LoRALinear

pd.set_option('display.max_colwidth', None)

# Set Device

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

print(f"Using device: {device}")

Using device: cpu


# Load VoxCeleb2 Dataset

In [5]:
# Load the vox celeb2 audio files path
vox2_path = '/Users/akanshagautam/Documents/MTech/Speech Understanding/Assignment 2/dataset/aac'
data = []

for root, dirs, files in os.walk(vox2_path):
    for file in files:
        if file.endswith(".m4a"):
            full_path = os.path.join(root, file)
            rel_path = os.path.relpath(full_path, os.path.join(vox2_path, "m4a"))
            parts = full_path.split(os.sep)
            identity = parts[parts.index('aac') + 1]
            data.append({"file_name": file, "abs_filepath": full_path, "rel_filepath": rel_path, "identity": identity})

df_vox2 = pd.DataFrame(data)
df_vox2 = df_vox2.sort_values(by="identity", ascending=True, ignore_index=True)
print(f"vox2 shape: {df_vox2.shape}")
df_vox2.head(2)

vox2 shape: (36237, 4)


,file_name,abs_filepath,rel_filepath,identity
0,00043.m4a,/Users/akanshagautam/Documents/MTech/Speech Understanding/Assignment 2/dataset/aac/id00017/OLguY5ofUrY/00043.m4a,../id00017/OLguY5ofUrY/00043.m4a,id00017
1,00179.m4a,/Users/akanshagautam/Documents/MTech/Speech Understanding/Assignment 2/dataset/aac/id00017/lvX8diWgriA/00179.m4a,../id00017/lvX8diWgriA/00179.m4a,id00017


In [12]:
# Load the waveform of voxCeleb2 audio files
waveforms = []
sampling_rates = []
for index, row in df_vox2.iterrows():
    abs_filepath = row['abs_filepath']
    try:
        waveform, sr = librosa.load(abs_filepath, sr=8000)
        waveforms.append(waveform)
        sampling_rates.append(sr)
    except Exception as e:
        print(f"Failed to load file: {abs_filepath}")
        waveforms.append([])
        sampling_rates.append(0)

df_vox2['waveform'] = waveforms
df_vox2['sampling_rate'] = sampling_rates

/var/folders/12/7zfx6db53p5bf6btn6s9bqy40000gn/T/ipykernel_24951/1820632893.py:7: UserWarning: PySoundFile failed. Trying audioread instead.
  waveform, sr = librosa.load(abs_filepath, sr=8000)
/Users/akanshagautam/Documents/MTech/ml_env/lib/python3.9/site-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


Failed to load file: /Users/akanshagautam/Documents/MTech/Speech Understanding/Assignment 2/dataset/aac/id02317/d4dinqezkKo/00383.m4a
Failed to load file: /Users/akanshagautam/Documents/MTech/Speech Understanding/Assignment 2/dataset/aac/id03030/haoNit7a4W0/00202.m4a
Failed to load file: /Users/akanshagautam/Documents/MTech/Speech Understanding/Assignment 2/dataset/aac/id03862/CJ8_3dVx7M4/00134.m4a
Failed to load file: /Users/akanshagautam/Documents/MTech/Speech Understanding/Assignment 2/dataset/aac/id04094/DRq5F2261Ko/00076.m4a
Failed to load file: /Users/akanshagautam/Documents/MTech/Speech Understanding/Assignment 2/dataset/aac/id04570/uLdhmrd9ULA/00369.m4a


In [14]:
# Split voxceleb2 dataset into train and test
all_speakers = sorted(df_vox2['identity'].unique())

train_speakers = all_speakers[:50]
test_speakers = all_speakers[50:100]
print("total train speakers:", len(train_speakers))
print("total test speakers:", len(test_speakers))

spk2label = {spk: i for i, spk in enumerate(train_speakers)}

df_train_vox2 = df_vox2[df_vox2['identity'].isin(train_speakers)].reset_index(drop=True)
df_test_vox2 = df_vox2[df_vox2['identity'].isin(test_speakers)].reset_index(drop=True)
print("train dataframe shape:", df_train_vox2.shape)
print("test datafram shape:", df_test_vox2.shape)

df_test_vox2.head(2)


total train speakers: 50
total test speakers: 50
train dataframe shape: (14979, 6)
test datafram shape: (14852, 6)


,file_name,abs_filepath,rel_filepath,identity,waveform,sampling_rate
0,00045.m4a,/Users/akanshagautam/Documents/MTech/Speech Understanding/Assignment 2/dataset/aac/id03677/KVUf0J2LAaA/00045.m4a,../id03677/KVUf0J2LAaA/00045.m4a,id03677,"[0.0083861165, 0.026233025, 0.034046598, 0.0303588, 0.013856893, 0.0012783967, 0.0017250986, 0.0048730876, -0.00033516577, -0.0056552067, -0.011302601, -0.01298482, -0.0021779481, 0.014832011, 0.026048947, 0.025501858, 0.020432718, 0.013180908, 0.002013075, -0.006209678, -0.007816484, -0.011034457, -0.017207533, -0.022063226, -0.022848576, -0.018088011, -0.007196877, 0.002857807, 0.0072496776, 0.0074041425, 0.009527016, 0.008918695, 0.0009219358, -0.0058669318, -0.0086981105, -0.0065184627, -0.0019151838, 0.0011819352, -0.0006137062, -0.0065999916, -0.008176668, -0.005478881, -0.006217077, -0.010039831, -0.0111991465, -0.009901179, -0.0076046647, -0.001035057, 0.0037649292, 0.0011443426, -0.0016406849, -0.0017831409, -0.0023553823, -0.0027396418, -0.00406816, -0.006611436, -0.007722936, -0.006516991, -0.0028938856, -0.0019445778, -0.0053353664, -0.009705443, -0.012247881, -0.008391787, -0.0030628918, -0.0026689013, -0.0044714846, -0.009046872, -0.014686999, -0.014612729, -0.010787064, -0.0069451327, -0.0028930919, -0.00074100774, -0.0014960687, -0.004205595, -0.0043240204, -0.0007527594, 0.0003778825, -0.0005301821, -0.0018669948, -0.0031750808, -0.0037565767, -0.0059486055, -0.00799782, -0.010575253, -0.012284346, -0.012557803, -0.01300272, -0.013386092, -0.008027082, 0.016552689, 0.06363653, 0.10263738, 0.09197521, 0.054114133, 0.01953137, -0.010047805, -0.018055957, -0.0061490694, ...]",8000
1,00051.m4a,/Users/akanshagautam/Documents/MTech/Speech Understanding/Assignment 2/dataset/aac/id03677/KVUf0J2LAaA/00051.m4a,../id03677/KVUf0J2LAaA/00051.m4a,id03677,"[0.019320436, 0.03655121, 0.029749721, 0.0116706, -0.017885588, -0.024872515, -0.016596453, -0.0136062, -0.012669281, -0.011029578, -0.010837223, -0.0020937575, 0.013294073, 0.018867452, 0.014649382, 0.009456221, 0.0040021315, 0.0024738002, 0.0030363374, -0.0028475039, -0.014277519, -0.020596761, -0.020448424, -0.014353655, -0.003434096, -0.00040759938, -0.0043857237, -0.007430923, -0.007788301, -0.0042911624, -0.0007943013, 0.0008188444, -0.008010548, -0.01835487, -0.017136555, -0.015836425, -0.017722221, -0.023662876, -0.025325205, -0.022349458, -0.01444138, -0.0067230733, -0.0035214273, -7.1565155e-05, 9.09213e-05, 0.0014040149, 0.0023844764, 0.0020964593, -0.0010533715, -0.0079608355, -0.014743929, -0.015604187, -0.012256578, -0.010190288, -0.012769461, -0.011529008, -0.009154839, -0.00827192, -0.010606424, -0.008814596, -0.009751826, -0.01079187, -0.007000606, -0.0061468165, -0.012010712, -0.016527511, -0.016534181, -0.027671956, -0.02806696, -0.036965776, -0.035271376, -0.022242937, 0.052599117, 0.1211664, 0.097171135, 0.073913544, 0.040370885, 0.012573782, 0.0143562155, 0.022682253, 0.002599637, -0.02502123, -0.021831624, -0.0073738196, 0.017710513, 0.027382646, 0.007982532, -0.0053475723, 0.00792577, 0.029529434, 0.034833577, 0.025220616, -0.008876369, -0.03850408, -0.035472564, -0.018637605, -0.0059719305, 0.0009182376, 0.0021455786, 0.008929301, ...]",8000


# Create a multi-speaker scenario dataset

In [32]:
num_samples = 1000
multi_speaker_data = []

for i in range(num_samples):
    sam1 = random.sample(df_train_vox2['identity'].unique().tolist(), 1)
    sam2 = random.sample(df_test_vox2['identity'].unique().tolist(), 1)

    x = df_train_vox2[df_train_vox2['identity'] == sam1[0]].sample(1).iloc[0]
    y = df_test_vox2[df_test_vox2['identity'] == sam2[0]].sample(1).iloc[0]

    wav1 = x['waveform']
    wav2 = y['waveform']

    min_len = min(len(wav1), len(wav2))
    mixed = wav1[:min_len] + wav2[:min_len]
    mixed_waveform =  mixed / np.max(np.abs(mixed))
    multi_speaker_data.append({
        "waveform": mixed_waveform,
        "x_waveform": wav1[:min_len],
        "y_waveform": wav2[:min_len],
        "x_identity": x['identity'],
        "y_identity": y['identity'],
        "x_abs_filepath": x['abs_filepath'],
        "y_abs_filepath": y['abs_filepath']
    })

df_mixed = pd.DataFrame(multi_speaker_data)
df_mixed.shape

(1000, 7)

# Task A: Use the pre-trained SepFormer model to perform speaker separation and speech enhancement of each speaker on the created test set by analyzing these metrics: Signal to Interference Ratio (SIR), Signal to Artefacts Ratio (SAR), Signal to Distortion Ratio (SDR) and Perceptual Evaluation of Speech Quality (PESQ).

## Perform Speaker Separation task on SepFormer model on the test set

In [13]:
# Load SepFormer model
model = separator.from_hparams(
    source="speechbrain/sepformer-whamr", 
    savedir='/Users/akanshagautam/Documents/MTech/Speech Understanding/Assignment 2/pretrained_models/sepformer-whamr'
)
model = model.to(device)

INFO:speechbrain.utils.fetching:Fetch hyperparams.yaml: Fetching from HuggingFace Hub 'speechbrain/sepformer-whamr' if not cached
INFO:speechbrain.utils.fetching:Fetch custom.py: Fetching from HuggingFace Hub 'speechbrain/sepformer-whamr' if not cached
INFO:speechbrain.utils.fetching:Fetch masknet.ckpt: Fetching from HuggingFace Hub 'speechbrain/sepformer-whamr' if not cached
INFO:speechbrain.utils.fetching:Fetch encoder.ckpt: Fetching from HuggingFace Hub 'speechbrain/sepformer-whamr' if not cached
INFO:speechbrain.utils.fetching:Fetch decoder.ckpt: Fetching from HuggingFace Hub 'speechbrain/sepformer-whamr' if not cached
INFO:speechbrain.utils.parameter_transfer:Loading pretrained files for: masknet, encoder, decoder


In [37]:
def extract_embedding(waveform):
    waveform = torch.tensor(waveform)
    if waveform.ndim > 1:
        waveform = waveform.mean(dim=0, keepdim=True)
    sources_separation = model.separate_batch(waveform.unsqueeze(0))
    return sources_separation.squeeze(0)

y_true = []
y_pred = []
similarities = []
metrics_data = []

for index, row in df_mixed.iterrows():
    mixed_waveform = torch.tensor(row['waveform']).unsqueeze(0).to(device)

    with torch.no_grad():
        estimated_sources = model.separate_batch(mixed_waveform)

    estimated_sources = estimated_sources.squeeze().cpu().numpy()
    true_source_x = row['x_waveform']
    true_source_y = row['y_waveform']

    true_sources = np.array([true_source_x, true_source_y])

    sdr, sir, sar, _ = bss_eval_sources(np.vstack(true_sources), np.vstack(estimated_sources))

    pesq_scores = [
        pesq(16000, true_sources[i], estimated_sources[i], 'wb')
        for i in range(len(true_sources))
    ]

    metrics_data.append({'SDR': np.mean(sdr), 'SIR': np.mean(sir), 'SAR': np.mean(sar), 'PESQ': np.mean(pesq_scores)})

metrics_data[0]


/var/folders/12/7zfx6db53p5bf6btn6s9bqy40000gn/T/ipykernel_24951/2317185987.py:25: FutureWarning: mir_eval.separation.bss_eval_sources
	Deprecated as of mir_eval version 0.8.
	It will be removed in mir_eval version 0.9.
  sdr, sir, sar, _ = bss_eval_sources(np.vstack(true_sources), np.vstack(estimated_sources))


ValueError: The shape of estimated sources and the true sources should match.  reference_sources.shape = (2, 39048), estimated_sources.shape = (39048, 2)

## Perform Speaker Enhancement task on SepFormer model on the test set